# **Generative LLM**

In [21]:
!pip install datasets
!pip install torch
!pip install transformers
!pip install peft
!pip install bitsandbytes -U
!pip install -U trl
!pip install nltk
!pip install faiss-cpu
!pip install sentence_transformers

  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached pyarrow-21.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached huggingface_hub-0.34.3-py3-none-any.whl.metadata (14 kB)
  Using cached hf_xet-1.1.5-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (879 bytes)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
Using cached huggingface_hub-0.34.3-py3-none-any.whl (558 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached p

In [1]:
import subprocess
import sys
from datasets import Dataset
import re
import string
import seaborn as sns
import os
import pandas as pd
import random
import torch 
import numpy as np
from datasets import Dataset
import bitsandbytes as bnb
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    GenerationConfig
)
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import warnings
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import faiss
import json
from tqdm import tqdm

warnings.filterwarnings('ignore')
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.empty_cache()

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **1) Import Data**

* Clean Linebreaks
* Limit Speech Length

In [5]:
data = pd.read_csv("../data/final_data.csv", on_bad_lines='skip')

# drop unnamed
if "Unnamed: 0" in data.columns:
    data = data.drop("Unnamed: 0", axis=1)

data["party"] = data["party"].replace("CDU/CSU", "Union")
data

,speech_text,legislative_period,protocol_nr,agenda_item_number,party,agenda_item_title,date,full_name
0,Herr Präsident! Kolleginnen und Kollegen! Die ...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Hans Michelbach
1,Sehr geehrter Herr Präsident! Liebe Kolleginne...,19,10,3,SPD,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Ingrid Arndt-Brauer
2,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Antje Tillmann
3,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,GRÜNE,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Gerhard Schick
4,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,19,10,3,FDP,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Florian Toncar
...,...,...,...,...,...,...,...,...
36112,Sehr geehrter Herr Präsident! Liebe Kolleginne...,20,99,5,SPD,Wolfsbestandsmanagement,2023-04-26,Lina Seitzl
36113,Frau Präsidentin! Meine Damen und Herren! Sie ...,20,99,3,GRÜNE,Aktuelle Stunde - Umstrittene Personalpolitik ...,2023-04-26,Till Steffen
36114,Sehr geehrter Herr Präsident! Meine Damen und ...,20,99,4,AfD,Bundeswehreinsatz Evakuierung aus Sudan,2023-04-26,Joachim Wundrak
36115,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,20,99,7,LINKE,Unregulierte Massenmigration,2023-04-26,Clara Bünger


In [6]:
# clean line breaks and special spaces
data["cleaned_text"] = data["speech_text"].apply(lambda x: x.replace("\xa0", " "))
data["cleaned_text"] = data["cleaned_text"].apply(lambda x: x.replace("\n", " "))

# remove repeated spaces
data["cleaned_text"] = data["cleaned_text"].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

In [7]:
# extract count of words
data["word_count"] = data["cleaned_text"].apply(lambda x: len(x.split()))

# limit speech length
data_filtered = data[data["word_count"] >= 200][data["word_count"] < 1501]

print("Nr. of Training Speeches:", len(data_filtered))
data_filtered

Nr. of Training Speeches: 36117


,speech_text,legislative_period,protocol_nr,agenda_item_number,party,agenda_item_title,date,full_name,cleaned_text,word_count
0,Herr Präsident! Kolleginnen und Kollegen! Die ...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Hans Michelbach,Herr Präsident! Kolleginnen und Kollegen! Die ...,520
1,Sehr geehrter Herr Präsident! Liebe Kolleginne...,19,10,3,SPD,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Ingrid Arndt-Brauer,Sehr geehrter Herr Präsident! Liebe Kolleginne...,693
2,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Antje Tillmann,Herr Präsident! Liebe Kolleginnen und Kollegen...,710
3,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,GRÜNE,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Gerhard Schick,Herr Präsident! Liebe Kolleginnen und Kollegen...,752
4,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,19,10,3,FDP,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Florian Toncar,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,872
...,...,...,...,...,...,...,...,...,...,...
36112,Sehr geehrter Herr Präsident! Liebe Kolleginne...,20,99,5,SPD,Wolfsbestandsmanagement,2023-04-26,Lina Seitzl,Sehr geehrter Herr Präsident! Liebe Kolleginne...,837
36113,Frau Präsidentin! Meine Damen und Herren! Sie ...,20,99,3,GRÜNE,Aktuelle Stunde - Umstrittene Personalpolitik ...,2023-04-26,Till Steffen,Frau Präsidentin! Meine Damen und Herren! Sie ...,789
36114,Sehr geehrter Herr Präsident! Meine Damen und ...,20,99,4,AfD,Bundeswehreinsatz Evakuierung aus Sudan,2023-04-26,Joachim Wundrak,Sehr geehrter Herr Präsident! Meine Damen und ...,611
36115,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,20,99,7,LINKE,Unregulierte Massenmigration,2023-04-26,Clara Bünger,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,480


In [8]:
# list of greeting-related words
greeting_words = [
    "Damen", "Herren", "Herr", "Kollegen", "Kolleginnen", "Präsident", "Präsidentin",
    "Kollege", "Kollegin", "verehrte", "verehrten", "geehrte", "geehrter", "geehrten"
]

# Build a regex pattern to match any of these words as whole words (case insensitive)
greeting_pattern = re.compile(r'\b(?:' + '|'.join(greeting_words) + r')\b', flags=re.IGNORECASE)

def remove_greeting_sentences(text, max_sentences=10):
    # Tokenize into sentences
    sentences = sent_tokenize(text, language='german')
    
    cleaned_sentences = []
    for i, sentence in enumerate(sentences):
        if i < max_sentences and greeting_pattern.search(sentence):
            continue  # Skip greeting sentence
        cleaned_sentences.append(sentence)
    
    return ' '.join(cleaned_sentences)

# apply
data_filtered['speech_text_cleaned'] = data_filtered['cleaned_text'].apply(remove_greeting_sentences)
data_filtered

,speech_text,legislative_period,protocol_nr,agenda_item_number,party,agenda_item_title,date,full_name,cleaned_text,word_count,speech_text_cleaned
0,Herr Präsident! Kolleginnen und Kollegen! Die ...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Hans Michelbach,Herr Präsident! Kolleginnen und Kollegen! Die ...,520,"Die Bankenunion ist einer von drei Bausteinen,..."
1,Sehr geehrter Herr Präsident! Liebe Kolleginne...,19,10,3,SPD,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Ingrid Arndt-Brauer,Sehr geehrter Herr Präsident! Liebe Kolleginne...,693,Liebe Gäste auf der Tribüne! Ich finde es schö...
2,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Antje Tillmann,Herr Präsident! Liebe Kolleginnen und Kollegen...,710,Auch ohne die FDP hat es dieser Deutsche Bunde...
3,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,GRÜNE,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Gerhard Schick,Herr Präsident! Liebe Kolleginnen und Kollegen...,752,"Ich glaube, es war deutlich, dass sich selbst ..."
4,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,19,10,3,FDP,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Florian Toncar,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,872,Das mag so sein. Aber wenn man sich in dieser ...
...,...,...,...,...,...,...,...,...,...,...,...
36112,Sehr geehrter Herr Präsident! Liebe Kolleginne...,20,99,5,SPD,Wolfsbestandsmanagement,2023-04-26,Lina Seitzl,Sehr geehrter Herr Präsident! Liebe Kolleginne...,837,"Mit der Redezeit, die ich habe, möchte ich ger..."
36113,Frau Präsidentin! Meine Damen und Herren! Sie ...,20,99,3,GRÜNE,Aktuelle Stunde - Umstrittene Personalpolitik ...,2023-04-26,Till Steffen,Frau Präsidentin! Meine Damen und Herren! Sie ...,789,"Sie hörten gerade die Rede eines Menschen, von..."
36114,Sehr geehrter Herr Präsident! Meine Damen und ...,20,99,4,AfD,Bundeswehreinsatz Evakuierung aus Sudan,2023-04-26,Joachim Wundrak,Sehr geehrter Herr Präsident! Meine Damen und ...,611,Die jüngsten Gewaltausbrüche im Sudan und dere...
36115,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,20,99,7,LINKE,Unregulierte Massenmigration,2023-04-26,Clara Bünger,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,480,"Ehrlich gesagt bin ich es leid, dass wir uns j..."


In [159]:
# sainity check
print(data_filtered["speech_text"][100])
print(data_filtered["speech_text_cleaned"][100])

Frau Präsidentin! Liebe Kolleginnen und Kollegen! Meine sehr verehrten Damen und Herren! Als ich 2013 ganz frisch im Bundestag die Berichterstattung für digitale Bildung übernommen habe, wusste mit diesen Begriffen noch kaum jemand etwas anzufangen, außer natürlich einer eingeschworenen Gemeinschaft digitalaffiner Lehrkräfte im Twitterlehrerzimmer. Seither sind wir wesentlich weitergekommen. Das freut mich ungemein. Nicht zuletzt waren es unser parlamentarischer Antrag im Jahr 2015 – danke noch einmal an den Kollegen Sven Volmering, der leider nicht mehr hier in unseren Reihen sitzt – und auch die beharrliche Aufklärungs- und Überzeugungsarbeit einer Community, die die KMK dazu bewogen haben, Handlungsempfehlungen für eine solche Bildungswelt zu erarbeiten. Dann kam die Ankündigung der damaligen Bildungsministerin Wanka eines DigitalPakts in der „BamS“. Jetzt haben wir den DigitalPakt, und zwar mit Brief und Siegel. Das ist großartig. Das ist ein Anlass zur Freude, zum Durchatmen, aber

### **Get Topics and Counts of Speeches per Party and Topic**

In [69]:
parties = data_filtered["party"].unique()

for party in parties:
    print(f"Partei: {party}")
    print(data_filtered[data_filtered["party"] == party]["agenda_item_title"].value_counts().head(20))


Partei: Union
agenda_item_title
Auswärtiges Amt                                                   75
Verteidigung                                                      68
Abschließende Beratungen ohne Aussprache                          67
Wirtschaftliche Zusammenarbeit und Entwicklung                    66
Gesundheit                                                        64
Ernährung und Landwirtschaft                                      58
Bildung und Forschung                                             53
Überweisungen im vereinfachten Verfahren                          51
Familie, Senioren, Frauen und Jugend                              49
Arbeit und Soziales                                               43
Bundeswehreinsatz in Kosovo (KFOR)                                35
Digitales und Verkehr                                             33
Bundeswehreinsatz in Mali (MINUSMA)                               32
Umwelt, Naturschutz und nukleare Sicherheit                       32
Bu

In [70]:
def classify_topic_from_agenda(title):
    if pd.isna(title):
        return "Sonstiges"

    title = str(title).lower()

    # 1
    if "mindestlohn" in title or "lohnerhöhung" in title or "arbeitslohn" in title:
        return "Mindestlohn"

    # 2
    elif "kosovo" in title or "kfor" in title or "bundeswehreinsatz im kosovo" in title or "einsatz der bundeswehr im kosovo" in title:
        return "Kosovo"

    # 3
    elif any(term in title for term in [
        "corona", "covid", "pandemie", "wirtschaftshilfe",
        "wirtschaftsstabilisierungsfonds", "coronahilfen",
        "pandemiehilfe",
    ]):
        return "Pandemiehilfen"

    # 4
    elif any(term in title for term in [
        "gas", "gaspreis", "energiepreis", "energiepreise",
        "gasversorgung", "gasmarkt", "energiekrise",
        "preisdeckel", "gaspreisbremse"
    ]):
        return "Gaspreise"

    else:
        return "Sonstiges"


In [71]:
data_filtered["topic_for_RAG"] = data_filtered["agenda_item_title"].apply(classify_topic_from_agenda)

topic_counts = data_filtered.groupby(["party", "topic_for_RAG"]).size().reset_index(name="count")
pivot_table = topic_counts.pivot(index="party", columns="topic_for_RAG", values="count").fillna(0).astype(int)
print(pivot_table)

# enough speeches for RAG

topic_for_RAG  Gaspreise  Kosovo  Mindestlohn  Pandemiehilfen  Sonstiges
party                                                                   
AfD                   55      18           14             144       4810
FDP                   63      15           15             124       4466
GRÜNE                 79      14           18             114       4842
LINKE                 41      13           11             105       3448
SPD                  112      27           27             209       7657
Union                108      38           31             275       9224


In [74]:
# save

topics = ["Mindestlohn", "Kosovo", "Pandemiehilfen", "Gaspreise"]
filtered_df = data_filtered[data_filtered["topic_for_RAG"].isin(topics)]

structured_speeches = []

for idx, row in filtered_df.iterrows():
    structured_speeches.append({
        "party": row["party"],
        "topic": row["topic_for_RAG"],
        "speech_nr": idx,
        "original_speech": row["cleaned_text"]
    })
    
with open("classified_speeches.json", "w", encoding="utf-8") as f:
    json.dump(structured_speeches, f, ensure_ascii=False, indent=2)


## **2) Sample Speeches**

* Ensure same number of training speeches per party

In [77]:
parties_count = data_filtered[["cleaned_text", "party"]].groupby("party").count().reset_index()
parties_count

,party,cleaned_text
0,AfD,5041
1,FDP,4683
2,GRÜNE,5067
3,LINKE,3618
4,SPD,8032
5,Union,9676


In [161]:
data_filtered = data_filtered.drop_duplicates(subset="cleaned_text")

In [162]:
# As we agreed to a balance sample, I shorten the dataframe to te maximmum length of LINKE
max_length = parties_count["cleaned_text"].min() #3618

parteien = data_filtered["party"].unique()

# Leerer DataFrame zum Sammeln
clean_df = pd.DataFrame()

# For every random subset if more than 3618 speeches, else take all of them
for partei in parteien:
    partei_unique_df = data_filtered[data_filtered["party"] == partei]

    if len(partei_unique_df) >= max_length:
        partei_sample = partei_unique_df.sample(n=max_length, random_state=42)
    else:
        partei_sample = partei_unique_df


    clean_df = pd.concat([clean_df, partei_sample], ignore_index=True)


clean_df.head()

,speech_text,legislative_period,protocol_nr,agenda_item_number,party,agenda_item_title,date,full_name,cleaned_text,word_count,speech_text_cleaned,topic_for_RAG
0,Sehr geehrte Frau Präsidentin! Werte Kolleginn...,20,79,12,Union,EU-Richtlinie Umweltauswirkungen Kunststoffpro...,2023-01-19,Björn Simon,Sehr geehrte Frau Präsidentin! Werte Kolleginn...,789,Auch von meiner Seite aus einen herzlichen Glü...,Sonstiges
1,Hochgeschätzter Herr Präsident! Kolleginnen un...,19,23,6,Union,Verkehr und digitale Infrastruktur,2018-03-22,Andreas Scheuer,Hochgeschätzter Herr Präsident! Kolleginnen un...,1292,Luftqualität ist Lebensqualität; aber Lebensqu...,Sonstiges
2,Frau Präsidentin! Verehrte Kolleginnen und Kol...,20,18,8,Union,Aktuelle Stunde - Bundeswehreinsätze in Mali b...,2022-02-18,Reinhard Brandl,Frau Präsidentin! Verehrte Kolleginnen und Kol...,741,Die Ampelkoalition kann jetzt nichts für die S...,Sonstiges
3,Frau Präsidentin! Liebe Kolleginnen und Kolleg...,20,21,4,Union,Meinungsfreiheit in Sozialen Medien,2022-03-17,Marc Henrichmann,Frau Präsidentin! Liebe Kolleginnen und Kolleg...,936,Das vierte Wort im Antrag der AfD ist „Russlan...,Sonstiges
4,Herr Präsident! Meine sehr verehrten Damen und...,19,164,3,Union,Bundeswehreinsatz EUTM Mali,2020-05-29,Johann David Wadephul,Herr Präsident! Meine sehr verehrten Damen und...,1011,Die Lage im Sahel ist kritisch. Sie ist sogar ...,Sonstiges


## **3.) Import the Base Model**

In [39]:
# Model configuration
model_name = "jphme/Llama-2-13b-chat-german"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading Llama model: {model_name}")
print(f"Device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)


# Load LLama model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    quantization_config = quantization_config,
    device_map="auto" if torch.cuda.is_available() else None,
)

print("Llama model and tokenizer loaded successfully!")
print(f"Model parameters: {base_model.num_parameters():,}")


Loading Llama model: jphme/Llama-2-13b-chat-german
Device: cuda


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Llama model and tokenizer loaded successfully!
Model parameters: 13,015,864,320


In [40]:
# create the chat template --> was missing

tokenizer.chat_template = """<s>[INST] <<SYS>>Du bist ein hilfreicher, respektvoller und ehrlicher Assistent.
Verhalte dich wie ein*e Abgeordnete*r im deutschen Bundestag.
Gegeben einer Parteizugehörigkeit und eines Themas, schreibe eine politische Rede im Umfang von 200 bis 1500 Worten.
<</SYS>>

{% for message in messages %}
{{ message['content'] }}{% if not loop.last %} [/INST] <s>[INST] {% else %} [/INST] {% endif %}
{% endfor %}</s>"""

tokenizer.model_max_length = 2048

## **4.) Generate a baseline speech given a title**

In [14]:
def generate_prediction_base(fraction_label, title, model, tokenizer):
    
    """Generate politicat party speeches using the base Llama model with chat template"""
    
    # Create messages in the format expected by Llama
    messages = [
        {
            "role": "user",
            "content": f" Du bist ein*e Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: {fraction_label}. Schreibe eine politische Rede mit bis zu 1500 Worten, die du im Bundestag hälst. Der Titel ist {title}."
        }
    ]

    # Apply chat template with thinking disabled for consistency
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # Disable for consistency with training
    )

    model_inputs = tokenizer([formatted_text], return_tensors="pt", max_length=2048, truncation=True).to(model.device)

    # Generate prediction with proper stopping
    with torch.inference_mode():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,  # Ensure EOS token stops generation
        )

    # Extract only the new tokens
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # Decode the response and remove EOS token if present
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return content

In [15]:
# get an actual speech title per party

random_title_info = {}

for party in clean_df["party"].unique():
    sample = clean_df[clean_df["party"] == party].sample(n=1, random_state=23)
    index = sample.index[0]
    title = sample["agenda_item_title"].values[0]
    random_title_info[party] = {"index": index, "title": title}

random_title_info

{'Union': {'index': 713, 'title': 'Regulierung von Bonitätsauskünften'},
 'SPD': {'index': 4331,
  'title': 'Bildung und Forschung für geflüchtete Ukrainer'},
 'GRÜNE': {'index': 7949,
  'title': 'Verteidigungspolitik, Einsatzbereitschaft Bundeswehr'},
 'FDP': {'index': 11567,
  'title': 'Aktuelle Stunde zur Eskalation in der Golfregion'},
 'AfD': {'index': 15185, 'title': 'Kinderzukunftsprogramm'},
 'LINKE': {'index': 18803, 'title': 'Netzwerkdurchsetzungsgesetz'}}

In [17]:
# make predictions for base model on title


prediction_base = []

for party, value in random_title_info.items():
    print(party)
    prediction_base.append(generate_prediction_base(party, value["title"], base_model, tokenizer))

Union
SPD
GRÜNE
FDP
AfD
LINKE


In [23]:
prediction_base_df = pd.DataFrame(prediction_base).rename(columns = {0 : "speech"})
prediction_base_df["party"] =  list(random_title_info.keys())
prediction_base_df.to_csv("../data/generative_predictions_base.csv")
prediction_base_df

,speech,party
0,#Regulierung von Bonitätsauskünften#\n\nHerr P...,Union
1,#Bildung und Forschung für geflüchtete Ukraine...,SPD
2,"#Verteidigungspolitik, Einsatzbereitschaft Bun...",GRÜNE
3,"#Halte 1#\n\nHerr Präsident, verehrtes Haus,\n...",FDP
4,"#Hallo, meine Damen und Herren,\n\nheute möcht...",AfD
5,#Netzwerkdurchsetzungsgesetz#\n\nHerr Präsiden...,LINKE


**Compare Predictions and true speeches** 

In [22]:
for i, (party, value) in enumerate(random_title_info.items()):
    
    # true speech
    true_speech = clean_df.loc[clean_df['agenda_item_title'] == value["title"], 'speech_text'].values[0]

    # Get the predicted speech
    predicted_speech = prediction_base[i]

    # Print comparison
    print("\n===================")
    print(f"=== {party} ===")
    print(f"Agenda Item: {value['title']}")
    print("===================")

    print(" True Speech:")
    print("===================")
    print(true_speech)
    
    print("Predicted Speech:")
    print("===================")
    print(predicted_speech)


=== Union ===
Agenda Item: Regulierung von Bonitätsauskünften
 True Speech:
Ich warte gern noch etwas, wenn Sie mich weiter loben wollen, Frau Präsidentin.
Sonntag ist der erste Advent.
Sehr geehrte Frau Präsidentin! Liebe Kolleginnen! Liebe Kollegen! In unserem Land besteht grundsätzlich die im Grundgesetz verankerte Vertragsfreiheit. Mit der hat eine Partei in diesem Haus immer wieder sichtliche Probleme. Lieber Matthias Birkwald, liebe Frau Nastic, heute Nachmittag haben wir hier über die Frage eines Kündigungsschutzes für über 70-jährige Mieter debattiert.
Das klingt toll, das klingt gut, aber das ist bei Ihren Anträgen immer so. Es gibt da immer zwei Seiten der Medaille, eine positive, die verlockend klingt – das ist der Kündigungsschutz für über 70-jährige Mieter –, aber gleichzeitig die Rückseite der Medaille, nämlich die Problematik, dass dann ein 65-jähriger, 67-jähriger, 68-jähriger Mietinteressent gar keinen Mietvertrag mehr bekommen wird.
Das heißt ja im Endeffekt: Was imm

## **5) Fine Tuning with Political Speeches**

In [24]:
# reload model for fine tuned


model_name = "jphme/Llama-2-13b-chat-german"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading Llama model: {model_name}")
print(f"Device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)


# Load LLama model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    quantization_config = quantization_config,
    device_map="auto" if torch.cuda.is_available() else None,
)

print("Llama model and tokenizer loaded successfully!")
print(f"Model parameters: {model.num_parameters():,}")


Loading Llama model: jphme/Llama-2-13b-chat-german
Device: cuda


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Llama model and tokenizer loaded successfully!
Model parameters: 13,015,864,320


In [28]:
# create the chat template --> was missing

tokenizer.chat_template = """<s>[INST] <<SYS>>Du bist ein hilfreicher, respektvoller und ehrlicher Assistent.
Verhalte dich wie ein*e Abgeordnete*r im deutschen Bundestag.
Gegeben einer Parteizugehörigkeit und eines Themas, schreibe eine politische Rede im Umfang von 200 bis 1500 Worten.
<</SYS>>

{% for message in messages %}
{{ message['content'] }}{% if not loop.last %} [/INST] <s>[INST] {% else %} [/INST] {% endif %}
{% endfor %}</s>"""

tokenizer.model_max_length = 2048

In [80]:
# convert data to suitable format
def row_to_prompt(row):

    # get correct style of prompt
    messages = [
        {
            "role": "user",
            "content": f" Du bist ein*e Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: {row['party']}. Schreibe eine politische Rede mit bis zu 1500 Worten, die du im Bundestag hälst. Der Titel ist {row['agenda_item_title']}."

        },
        {
            "role": "assistant",
            "content": row['cleaned_text']
        }
    ]

    # apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return prompt

In [81]:
# make sataset 

clean_df["text"] = clean_df.apply(row_to_prompt, axis=1)
dataset = Dataset.from_pandas(clean_df[["text"]])

In [83]:
clean_df['text'][0]

'<s>[INST] <<SYS>>Du bist ein hilfreicher, respektvoller und ehrlicher Assistent.\nVerhalte dich wie ein*e Abgeordnete*r im deutschen Bundestag.\nGegeben einer Parteizugehörigkeit und eines Themas, schreibe eine politische Rede im Umfang von 200 bis 1500 Worten.\n<</SYS>>\n\n Du bist ein*e Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: Union. Schreibe eine politische Rede mit bis zu 1500 Worten, die du im Bundestag hälst. Der Titel ist EU-Richtlinie Umweltauswirkungen Kunststoffprodukte. [/INST] <s>[INST] Sehr geehrte Frau Präsidentin! Werte Kolleginnen und Kollegen! Auch von meiner Seite aus einen herzlichen Glückwunsch an die Ministerin und alles Gute für das neue Lebensjahr. Liebe Kolleginnen und Kollegen, keine Frage: Achtlos weggeworfene Kunststoffprodukte stellen ein großes Problem für unsere Umwelt dar; das wissen wir alle. Sie benötigen Jahrzehnte, wenn nicht gar Jahrhunderte, um sich zu zersetzen, belasten unsere Ökosysteme und schaden uns Menschen und auch der Tier

In [84]:
# tokenize the resulting chat formated prompt
def tokenize(example):
    result = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=2048,
    )
    # Labels for causal LM training are the input_ids themselves
    result["labels"] = result["input_ids"].copy()
    return result

In [33]:
tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/21708 [00:00<?, ? examples/s]

In [34]:
# training settings

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./llama2-all-speeches",
    per_device_train_batch_size=1, 
    gradient_accumulation_steps=8, 
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)



In [35]:
trainer = SFTTrainer(
    model=model, # fine tune the model
    train_dataset=tokenized_dataset, # not sample,
    args=training_args,
    peft_config=lora_config
)

trainer.train()
trainer.save_model("./Llama_all_speeches")


Truncating train dataset:   0%|          | 0/21708 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,14.559400
20,5.529500
30,2.612400
40,2.391100
50,1.666600
60,1.095400
70,0.755800
80,0.606400
90,0.592400
100,0.559900


In [37]:
from transformers import GenerationConfig

# Set a valid config for deterministic generation
gen_config = GenerationConfig(
    do_sample=False,
    temperature=None,
    top_p=None
)

model.generation_config = gen_config

# Now save --> issue at reimport, apparently only the base model weights were saved... Later be restored over saved trainer
model.save_pretrained("generator_ft/")
tokenizer.save_pretrained("generator_ft/")

('generator_ft/tokenizer_config.json',
 'generator_ft/special_tokens_map.json',
 'generator_ft/chat_template.jinja',
 'generator_ft/tokenizer.json')

In [38]:

def generate_prediction_after_ft(fraction_label, title, model, tokenizer, temp):
    """Generate politicat party speeches using the base Llama model with chat template"""
    # Create messages in the format expected by Llama
    messages = [
        {
            "role": "user",
            "content": f" Du bist ein*e Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: {fraction_label}. Schreibe eine politische Rede mit bis zu 1500 Worten, die du im Bundestag hälst. Der Titel ist {title}."
        }
    ]

    # Apply chat template with thinking disabled for consistency
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # Disable for consistency with training
    )

    model_inputs = tokenizer([formatted_text], return_tensors="pt", max_length=2048, truncation=True).to(model.device)


    # Generate prediction with proper stopping
    with torch.inference_mode():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048,
            temperature=temp,
            repetition_penalty = 1.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,  # Ensure EOS token stops generation            
        )

    # Extract only the new tokens
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # Decode the response and remove EOS token if present
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return content

In [39]:
# temperature at 0.1

prediction_after_finetune_01 = []

for party, value in random_title_info.items():
    print(party)
    print(value["title"])
    prediction_after_finetune_01.append(generate_prediction_after_ft(party, value["title"], model, tokenizer, temp = 0.1))

Union
Regulierung von Bonitätsauskünften
SPD
Bildung und Forschung für geflüchtete Ukrainer
GRÜNE
Verteidigungspolitik, Einsatzbereitschaft Bundeswehr
FDP
Aktuelle Stunde zur Eskalation in der Golfregion
AfD
Kinderzukunftsprogramm
LINKE
Netzwerkdurchsetzungsgesetz


In [40]:
# temperature at 0.3

prediction_after_finetune_03 = []

for party, value in random_title_info.items():
    print(party)
    print(value["title"])
    prediction_after_finetune_03.append(generate_prediction_after_ft(party, value["title"], model, tokenizer, temp = 0.3))

Union
Regulierung von Bonitätsauskünften
SPD
Bildung und Forschung für geflüchtete Ukrainer
GRÜNE
Verteidigungspolitik, Einsatzbereitschaft Bundeswehr
FDP
Aktuelle Stunde zur Eskalation in der Golfregion
AfD
Kinderzukunftsprogramm
LINKE
Netzwerkdurchsetzungsgesetz


In [44]:
# export

prediction_after_finetune_01_df = pd.DataFrame(prediction_after_finetune_01).rename(columns = {0 : "speech"})
prediction_after_finetune_01_df["party"] =  list(random_title_info.keys())
prediction_after_finetune_01_df.to_csv("../data/generative_prediction_after_finetune_01.csv")
prediction_after_finetune_01_df

prediction_after_finetune_03_df = pd.DataFrame(prediction_after_finetune_03).rename(columns = {0 : "speech"})
prediction_after_finetune_03_df["party"] =  list(random_title_info.keys())
prediction_after_finetune_03_df.to_csv("../data/generative_prediction_after_finetune_03.csv")
prediction_after_finetune_03_df

,speech,party
0,# INFOS SEITE DU BIST EIN HILFEZWEIG FÜR SICHE...,Union
1,[INST] Sehr geehrte Frau Präsidentin! Liebe Ko...,SPD
2,#INCLUDE <https://www.youtube-nocookie.com/_Yl...,GRÜNE
3,#NeverAgain – so lautet das Motto des jüngsten...,FDP
4,[_] Sehr geehrte Frau Präsidentin! Verehrte Ko...,AfD
5,[INST] Sehr geehrte Frau Präsidentin! Liebe Ko...,LINKE


In [43]:
# compare

for i, (party, value) in enumerate(random_title_info.items()):
    
    # true speech
    true_speech = clean_df.loc[clean_df['agenda_item_title'] == value["title"], 'speech_text'].values[0]

    # Get the predicted speech
    predicted_speech = prediction_base[i]
    finetuned_speech_01 = prediction_after_finetune_01[i]
    finetuned_speech_03 = prediction_after_finetune_03[i]

    # Print comparison
    print("==================")
    print(f"\n=== {party} ===")
    print(f"Agenda Item: {value['title']}\n")
    print("==================")
    
    print("True Speech:\n")
    print("==================")
    print(true_speech)
    
    print("Initial predicted Speech:\n")
    print("==================")
    print(predicted_speech)
    
    print("Fine-tuned predicted (0.1 temp) Speech:\n")
    print("==================")
    print(finetuned_speech_01)
    
    print("Fine-tuned predicted (0.3 temp) Speech:\n")
    print("==================")
    print(finetuned_speech_03)


=== Union ===
Agenda Item: Regulierung von Bonitätsauskünften

True Speech:

Ich warte gern noch etwas, wenn Sie mich weiter loben wollen, Frau Präsidentin.
Sonntag ist der erste Advent.
Sehr geehrte Frau Präsidentin! Liebe Kolleginnen! Liebe Kollegen! In unserem Land besteht grundsätzlich die im Grundgesetz verankerte Vertragsfreiheit. Mit der hat eine Partei in diesem Haus immer wieder sichtliche Probleme. Lieber Matthias Birkwald, liebe Frau Nastic, heute Nachmittag haben wir hier über die Frage eines Kündigungsschutzes für über 70-jährige Mieter debattiert.
Das klingt toll, das klingt gut, aber das ist bei Ihren Anträgen immer so. Es gibt da immer zwei Seiten der Medaille, eine positive, die verlockend klingt – das ist der Kündigungsschutz für über 70-jährige Mieter –, aber gleichzeitig die Rückseite der Medaille, nämlich die Problematik, dass dann ein 65-jähriger, 67-jähriger, 68-jähriger Mietinteressent gar keinen Mietvertrag mehr bekommen wird.
Das heißt ja im Endeffekt: Was im

In [ ]:
# reimport data
prediction_base_df = pd.read_csv("../data/generative_predictions_base.csv").drop(columns = "Unnamed: 0")
prediction_after_finetune_01_df = pd.read_csv("../data/generative_prediction_after_finetune_01.csv").drop(columns = "Unnamed: 0")
prediction_after_finetune_03_df = pd.read_csv("../data/generative_prediction_after_finetune_03.csv").drop(columns = "Unnamed: 0")

# rename speeches
prediction_base_df = prediction_base_df.rename(columns = {"speech" : "speech_base"})
prediction_after_finetune_01_df = prediction_after_finetune_01_df.rename(columns = {"speech" : "speech_ft_01"})
prediction_after_finetune_03_df = prediction_after_finetune_03_df.rename(columns = {"speech" : "speech_ft_03"})
prediction_after_finetune_03_df

# merge to one df
speeches_df = (
    prediction_base_df
    .merge(prediction_after_finetune_01_df, on="party")
    .merge(prediction_after_finetune_03_df, on="party")
)

speeches_df.to_csv("../data/test_speeches_by_title.csv", index = False)
speeches_df

***
***

## **6) Speeches on Topics before RAG**

In [2]:
# REIMPORT 

model_name = "jphme/Llama-2-13b-chat-german"

device = "cuda" if torch.cuda.is_available() else "cpu"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "./Llama_all_speeches")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [3]:
# Merge LoRA adapters into the base model
merged_model = model.merge_and_unload()

# Define a new, valid generation config for deterministic generation
gen_config = GenerationConfig(
    do_sample=False  # sampling disabled, all other sampling params ignored
)

# Assign this new config before saving
merged_model.generation_config = gen_config

# Save merged model, tokenizer, and clean generation config
merged_model.save_pretrained("generator_final/")
tokenizer.save_pretrained("generator_final/")
gen_config.save_pretrained("generator_final/")  # optional, but explicit

In [35]:
def generate_speech_to_topic(fraction_label, topic, model, tokenizer, temp):
    
    """Generate politicat party speeches using the base Llama model with chat template"""
    # Create messages in the format expected by Llama
    messages = [
        {
            "role": "user",
            "content": f" Du bist ein*e Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: {fraction_label}. Schreibe eine politische Rede mit bis zu 1500 Worten, die du im Bundestag hälst. Es soll um das Thema {topic} gehen."
        }
    ]

    # Apply chat template with thinking disabled for consistency
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # Disable for consistency with training
    )

    model_inputs = tokenizer([formatted_text], return_tensors="pt", max_length=2048, truncation=True).to(model.device)


    # Generate prediction with proper stopping
    with torch.inference_mode():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048,
            temperature=temp,
            repetition_penalty = 1.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,  # Ensure EOS token stops generation            
        )

    # Extract only the new tokens
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # Decode the response and remove EOS token if present
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return content

In [53]:
# dictionary of party and topics

topic_to_speech = {"Union" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "SPD" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "GRÜNE" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "FDP" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "AfD" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "Linke" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"]}

In [41]:
# apply generate_speech_to_topic to base_model and model (ft) --> Temperature = 0.3

base_model_topic = {}
ft_model_topic = {}



for party, topic in topic_to_speech.items():
    print(party)
    base_model_topic[party] = []
    ft_model_topic[party] = []
    
    for top in topic:
        base_model_topic[party].append(generate_speech_to_topic(party, top, base_model, tokenizer, temp = 0.3))
        ft_model_topic[party].append(generate_speech_to_topic(party, top, model, tokenizer, temp = 0.3))


# save
data_rows = []

for party in base_model_topic:
    topics = topic_to_speech[party]
    base_speeches = base_model_topic[party]
    ft_speeches = ft_model_topic[party]
    
    for topic, base_speech, ft_speech in zip(topics, base_speeches, ft_speeches):
        data_rows.append({
            "party": party,
            "topic": topic,
            "speech_base_model": base_speech,
            "speech_ft_model": ft_speech
        })

# Create DataFrame
speeches_topic_df_03 = pd.DataFrame(data_rows)
speeches_topic_df_03.to_csv("../data/speeches_by_topic_03.csv", index = False)
speeches_topic_df_03

Union
SPD
GRÜNE
FDP
AfD
Linke


,party,topic,speech_base_model,speech_ft_model
0,Union,Mindestlohn,#HartzIV - Die Zeit der Verantwortung für alle...,# Hochwertige Arbeit für alle – Ein guter Job ...
1,Union,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieJamaisKrieg – so lautet der Slogan des Fri...
2,Union,Wirtschaftshilfen Corona,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – wir müssen lernen aus den Fehlents...
3,Union,Gaspreise,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! Wir werden unsere Energie unabhängi...
4,SPD,Mindestlohn,#HartzIV - Die soziale Marktwirtschaft braucht...,#NieAufKeinenFall – Das muss endlich Stoppen! ...
5,SPD,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin hier heute als Vertreterin der...,#NieAgain! Kein Krieg mehr für Deutschland – K...
6,SPD,Wirtschaftshilfen Corona,#WirHelfendeNation# - Eine solide wirtschaftli...,#NieAgain! – Das war der Slogan des Protestes ...
7,SPD,Gaspreise,#WirfürEnergie - Eine nachhaltige Energiesyste...,#NieAgain – wir müssen endlich den Krieg in de...
8,GRÜNE,Mindestlohn,"#HartzIV#Mindesteinkommen#ArbeitslosengeldII#,...",#NieMindeloesung! Das hat der Kollege Birkwald...
9,GRÜNE,Bundeswehreinsatz im Kosovo,"#Bundeswehr#Kosovokrieg"" class=""tw-breadcrumbs...",#NieWiederKosovowar! – Das war der Aufruf des ...


In [55]:
# apply generate_speech_to_topic to base_model and model (ft) --> Temperature = 0.1

base_model_topic_01 = {}
ft_model_topic_01 = {}



for party, topic in topic_to_speech.items():
    print(party)
    base_model_topic_01[party] = []
    ft_model_topic_01[party] = []
    
    for top in topic:
        base_model_topic_01[party].append(generate_speech_to_topic(party, top, base_model, tokenizer, temp = 0.1))
        ft_model_topic_01[party].append(generate_speech_to_topic(party, top, model, tokenizer, temp = 0.1))


data_rows = []

for party in base_model_topic_01:
    topics = topic_to_speech[party]
    base_speeches = base_model_topic_01[party]
    ft_speeches = ft_model_topic_01[party]
    
    for topic, base_speech, ft_speech in zip(topics, base_speeches, ft_speeches):
        data_rows.append({
            "party": party,
            "topic": topic,
            "speech_base_model": base_speech,
            "speech_ft_model": ft_speech
        })

# Create DataFrame
speeches_topic_df_01 = pd.DataFrame(data_rows)
speeches_topic_df_01.to_csv("../data/speeches_by_topic_01.csv", index = False)
speeches_topic_df_01

Union
SPD
GRÜNE
FDP
AfD
Linke


,party,topic,speech_base_model,speech_ft_model
0,Union,Mindestlohn,#Hartz-IV# - Ein neuer Start für den Arbeitsma...,#NieAgain – Sexuelle Gewalt an Kindern verhind...
1,Union,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieWieder – Das war der Slogan des Protestes ...
2,Union,Wirtschaftshilfen Corona,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – so lautet der Slogan des internati...
3,Union,Gaspreise,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! – Das war der Slogan des großen Pro...
4,SPD,Mindestlohn,"#HartzIV#Mindesteinkommen#, Ladies and Gentlem...",#NieAufKeinenFall! – Das war der Aufruf des Bu...
5,SPD,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,import { default as Spinner } from 'react-spin...
6,SPD,Wirtschaftshilfen Corona,#WirHelfendeNation# - Eine solide wirtschaftli...,#NieAgain – Das Ziel der Bundesregierung muss ...
7,SPD,Gaspreise,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,#NieAgain! Wir müssen den Krieg in der Ukraine...
8,GRÜNE,Mindestlohn,#Mindesteinkommen für alle!# Ein neuer Start i...,"#Mindesteinkommen#Minijob#, wir alle wissen es..."
9,GRÜNE,Bundeswehreinsatz im Kosovo,#Bundeswehr#Kosovokrieg#Politikdebatte#GrüneFr...,#NieWieder! – Das war der Aufruf des Klimaprot...


In [45]:
# compare 
speeches_topic_df_01 = pd.read_csv("../data/speeches_by_topic_01.csv")
speeches_topic_df_03 = pd.read_csv("../data/speeches_by_topic_03.csv")

print(speeches_topic_df_01["speech_ft_model"][1])
print(speeches_topic_df_03["speech_ft_model"][1])

#NieWieder – Das war der Slogan des Protestes am Wochenende in Berlin gegen den NATO-Einsatz KFOR nach dem völkerrechtswidrigem Angriff Russlands auf die Ukraine Ende Februar dieses Jahres; er sollte auch uns hier heute als Erinnerung dienen! Die Lage an unserer Grenze hat sich dramatisch verändert seitdem. Wir haben einen Krieg vor Augen, bei dessen Ausgang wir nicht wissen können oder wollen was passieren wird. Deshalb müssen alle Maßnahmen zur Stärkung deutliche Signale senden für Friedenssicherheit sowohl innerhalb Europas als auch jenseits seiner Grenzen. Der Einsatz deutscher Soldaten unter Führung Deutschlands beim international bewaffneten Eingriffe (Kosovokrieg) gehört dazu. Ich bin froh darüber, dass es gelungen ist, diesen Beschluss trotz aller Schwierigkeiten durchzusetzen. Denn Deutschland muss seine Verantwortlichkeit gegenüber seinen Partnern ausüben sowie sein Engagement zeigen, damit Europa zusammenhält. Gerade weil diese Situation so gefährlich ist, brauchten wir kein

***
***
# **RAG**

In [110]:
def build_faiss_index(texts, embedder):

    # Create FAISS index from a list of texts using a sentence embedding model:
    # determined which three of the possible speeches are the most similar to our desired topic

    embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index, embeddings


In [174]:
def truncate_prompt_to_token_limit(prompt, tokenizer, max_prompt_tokens):
    tokens = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    input_ids = tokens["input_ids"][0]
    if len(input_ids) <= max_prompt_tokens:
        return prompt  # no truncation needed

    # Truncate text by reducing its character length until it fits
    # This is a rough but fast approximation
    words = prompt.split()
    while len(tokenizer(" ".join(words), return_tensors="pt")["input_ids"][0]) > max_prompt_tokens:
        words = words[:-10]  # remove 10 tokens at a time
    return " ".join(words)


def generate_single_rag_speech(party, topic, texts, model, tokenizer, embedder, index, k=3, temp = 0.1):
    """
    Generate a single RAG-based speech for a party and topic, using a FAISS index.
    """
    query_vec = embedder.encode([topic])
    D, I = index.search(query_vec, k) # get the three most semantically similar speeches
    context_speeches = [texts[i] for i in I[0]]
    context = "\n---\n".join(context_speeches)

    prompt = [
            {
              "role": "user",
              "content": f"Schreibe eine neue, eigenständige Rede als Abgeordnete*r der Partei {party} zum Thema {topic}. Nutze die folgenden Reden als Kontext, aber formuliere die Rede in deinen eigenen Worten. Die Rede soll im Bundestag gehalten werden und bis zu 1500 Wörter lang sein.\n Kontext:\n{context}"
        
    }

    ]
    
    formatted_prompt = tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    
    print(f"Prompt length (tokens): {len(tokenizer(formatted_prompt)['input_ids'])}")



    MAX_PROMPT_TOKENS = 4096 - 1024
    formatted_text = truncate_prompt_to_token_limit(formatted_prompt, tokenizer, MAX_PROMPT_TOKENS)
    
    model_inputs = tokenizer(formatted_text, return_tensors="pt").to(model.device)

    #print(f"[DEBUG] Final prompt token count: {model_inputs['input_ids'].shape[1]}")
    #print(f"[DEBUG] Prompt:\n{formatted_text[:1000]}...")
    
    
    # Generate prediction with proper stopping
    with torch.inference_mode():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=1024,
            min_new_tokens = 100, # force the model to generate sth
            temperature=temp,
            repetition_penalty = 1.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,  # Ensure EOS token stops generation            
        )

    # Extract only the new tokens
    #output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    #print(f"[DEBUG] Generated token count: {len(output_ids)}")

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
    #print("Output token IDs:", output_ids)
    #print("Decoded output:", tokenizer.decode(output_ids))
    # Decode the response and remove EOS token if present
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return content, context_speeches



In [175]:

def generate_all_rag_speeches(clean_df, parties, topics, model, tokenizer, embedder, temp = 0.1):
    """
    Iterate over all party-topic combinations and generate one RAG-based speech for each.
    """
    results = []

    for party in tqdm(parties, desc="Parteien"):
        for topic in topics:
            subset = clean_df[(clean_df["party"] == party) & (clean_df["topic_for_RAG"] == topic)]

            texts = subset["speech_text_cleaned"].tolist()
            index, _ = build_faiss_index(texts, embedder)

            generated, contexts = generate_single_rag_speech(
                party=party,
                topic=topic,
                texts=texts,
                model=model,
                tokenizer=tokenizer,
                embedder=embedder,
                index=index,
                temp = temp
            )

            results.append({
                "party": party,
                "topic": topic,
                "speech_nr": 1,
                "generated_speech": generated,
                "context_speeches": contexts
            })

    return results


In [144]:
parties = ["SPD", "Union", "LINKE", "AfD", "GRÜNE", "FDP"]
topics = ["Mindestlohn", "Kosovo", "Pandemiehilfen", "Gaspreise"]

# Load embedding model
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Run generation
results_01 = generate_all_rag_speeches(clean_df, parties, topics, model, tokenizer, embedder, temp = 0.1)

with open("../data/rag_generated_speeches_final_01.json", "w", encoding="utf-8") as f:
    json.dump(results_01, f, ensure_ascii=False, indent=2)


Parteien:   0%|          | 0/6 [00:00<?, ?it/s]

Prompt length (tokens): 2770
Prompt length (tokens): 3829
Prompt length (tokens): 3716
Prompt length (tokens): 3499


Parteien:  17%|█▋        | 1/6 [02:36<13:04, 156.82s/it]

Prompt length (tokens): 4336
Prompt length (tokens): 2826
Prompt length (tokens): 4002
Prompt length (tokens): 3991


Parteien:  33%|███▎      | 2/6 [05:14<10:28, 157.06s/it]

...

Parteien:  67%|██████▋   | 4/6 [10:26<05:12, 156.49s/it]

Prompt length (tokens): 2674
Prompt length (tokens): 3494
Prompt length (tokens): 3001
Prompt length (tokens): 4486


Parteien:  83%|████████▎ | 5/6 [13:02<02:36, 156.41s/it]

Prompt length (tokens): 3817
Prompt length (tokens): 3488
Prompt length (tokens): 2755
Prompt length (tokens): 3915


Parteien: 100%|██████████| 6/6 [15:39<00:00, 156.52s/it]


In [46]:
# rename
speeches_topic_df_01 = speeches_topic_df_01.rename(columns = {"speech_base_model" : "speech_base_model_01", "speech_ft_model" : "speech_ft_model_01"})
speeches_topic_df_03 = speeches_topic_df_03.rename(columns = {"speech_base_model" : "speech_base_model_03", "speech_ft_model" : "speech_ft_model_03"})

# save RAG, temp 0,1
RAG_results_01 = pd.DataFrame(results_01)[["party", "topic", "generated_speech"]].rename(columns = {"generated_speech" : "RAG_speech_01"})
RAG_results_01

In [177]:
parties = ["SPD", "Union", "LINKE", "AfD", "GRÜNE", "FDP"]
topics = ["Mindestlohn", "Kosovo", "Pandemiehilfen", "Gaspreise"]

# Load embedding model
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Run generation
results_03 = generate_all_rag_speeches(clean_df, parties, topics, model, tokenizer, embedder, temp = 0.3)

Parteien:   0%|          | 0/6 [00:00<?, ?it/s]

Prompt length (tokens): 2770
Prompt length (tokens): 3829
Prompt length (tokens): 3716
Prompt length (tokens): 3499


Parteien:  17%|█▋        | 1/6 [02:36<13:03, 156.64s/it]

Prompt length (tokens): 4336
Prompt length (tokens): 2826
Prompt length (tokens): 4002
Prompt length (tokens): 3991


Parteien:  33%|███▎      | 2/6 [05:13<10:27, 156.99s/it]

...

Parteien:  67%|██████▋   | 4/6 [10:25<05:12, 156.45s/it]

Prompt length (tokens): 2674
Prompt length (tokens): 3494
Prompt length (tokens): 3001
Prompt length (tokens): 4486


Parteien:  83%|████████▎ | 5/6 [13:02<02:36, 156.36s/it]

Prompt length (tokens): 3817
Prompt length (tokens): 3488
Prompt length (tokens): 2755
Prompt length (tokens): 3915


Parteien: 100%|██████████| 6/6 [15:38<00:00, 156.47s/it]


In [178]:
with open("../data/rag_generated_speeches_final_03.json", "w", encoding="utf-8") as f:
    json.dump(results_03, f, ensure_ascii=False, indent=2)

In [6]:
# save RAG, temp 0,3
RAG_results_03 = pd.DataFrame(results_03)[["party", "topic", "generated_speech"]].rename(columns = {"generated_speech" : "RAG_speech_03"})
RAG_results_03

,party,topic,RAG_speech_03
0,SPD,Mindestlohn,[Inst] Sehr verehrte Präsidentin! Meine geschä...
1,SPD,Kosovo,damit maßgeblich zur Unterbindung potentieller...
2,SPD,Pandemiehilfen,Landes sowie seiner Gesellschaft trotz schwers...
3,SPD,Gaspreise,"wir nun begreifen, und darüber hinaus wollen w..."
4,Union,Mindestlohn,kurz kommentieren. Es geht dort tatsächlich um...
5,Union,Kosovo,[Inst] Sehr verehrte Frau Präsident! Lieber Ko...
6,Union,Pandemiehilfen,Bildungsfördermittel überzeugt gewesen. Doch d...
7,Union,Gaspreise,Kommission diesen Decker durchsetzten sowie we...
8,LINKE,Mindestlohn,[INSP_IPTV #Wirecard - /TWCMS/Bild/_Apfotosyst...
9,LINKE,Kosovo,"[Inst] Herr Dr. Hahnemann, lieber Peter, du ha..."


In [47]:
# recode 
RAG_results_01["topic"] = RAG_results_01["topic"].replace({"Kosovo" : "Bundeswehreinsatz im Kosovo", "Pandemiehilfen": "Wirtschaftshilfen Corona"})
RAG_results_03["topic"] = RAG_results_03["topic"].replace({"Kosovo" : "Bundeswehreinsatz im Kosovo", "Pandemiehilfen": "Wirtschaftshilfen Corona"})

speeches_topic_df_01["party"] = speeches_topic_df_01["party"].replace({"Linke" : "LINKE"})
speeches_topic_df_03["party"] = speeches_topic_df_03["party"].replace({"Linke" : "LINKE"})

In [16]:
# combine all final speeches and export ---> missing context

#RAG_speeches_df = (
#    speeches_topic_df_01
#    .merge(speeches_topic_df_03, on=["party", "topic"], how = "inner")
#    .merge(RAG_results_01, on=["party", "topic"], how = "inner")
#    .merge(RAG_results_03, on=["party", "topic"], how = "inner")
#)

#RAG_speeches_df.to_csv("../data/generated_speeches_final.csv", index = False)
#RAG_speeches_df

,party,topic,speech_base_model_01,speech_ft_model_01,speech_base_model_03,speech_ft_model_03,RAG_speech_01,RAG_speech_03
0,Union,Mindestlohn,#Hartz-IV# - Ein neuer Start für den Arbeitsma...,#NieAgain – Sexuelle Gewalt an Kindern verhind...,#HartzIV - Die Zeit der Verantwortung für alle...,# Hochwertige Arbeit für alle – Ein guter Job ...,kurz kommentieren. Es geht dort tatsächlich um...,kurz kommentieren. Es geht dort tatsächlich um...
1,Union,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieWieder – Das war der Slogan des Protestes ...,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieJamaisKrieg – so lautet der Slogan des Fri...,[Inst] Sehr verehrte Frau Präsidentin! Lieber ...,[Inst] Sehr verehrte Frau Präsident! Lieber Ko...
2,Union,Wirtschaftshilfen Corona,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – so lautet der Slogan des internati...,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – wir müssen lernen aus den Fehlents...,"Bildungsfördersprechanbieter, also Förster/-in...",Bildungsfördermittel überzeugt gewesen. Doch d...
3,Union,Gaspreise,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! – Das war der Slogan des großen Pro...,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! Wir werden unsere Energie unabhängi...,"Kommission versuchen, diesen Decker möglichst ...",Kommission diesen Decker durchsetzten sowie we...
4,SPD,Mindestlohn,"#HartzIV#Mindesteinkommen#, Ladies and Gentlem...",#NieAufKeinenFall! – Das war der Aufruf des Bu...,#HartzIV - Die soziale Marktwirtschaft braucht...,#NieAufKeinenFall – Das muss endlich Stoppen! ...,[Inst] Sehr verehrte Präsidentin! Meine geschä...,[Inst] Sehr verehrte Präsidentin! Meine geschä...
5,SPD,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,import { default as Spinner } from 'react-spin...,#Hallo! Ich bin hier heute als Vertreterin der...,#NieAgain! Kein Krieg mehr für Deutschland – K...,damit maßgeblich zur Unterbindung potentieller...,damit maßgeblich zur Unterbindung potentieller...
6,SPD,Wirtschaftshilfen Corona,#WirHelfendeNation# - Eine solide wirtschaftli...,#NieAgain – Das Ziel der Bundesregierung muss ...,#WirHelfendeNation# - Eine solide wirtschaftli...,#NieAgain! – Das war der Slogan des Protestes ...,Landes sowie seine Standortattraktivität erhal...,Landes sowie seiner Gesellschaft trotz schwers...
7,SPD,Gaspreise,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,#NieAgain! Wir müssen den Krieg in der Ukraine...,#WirfürEnergie - Eine nachhaltige Energiesyste...,#NieAgain – wir müssen endlich den Krieg in de...,wir fortsetzten. Zum Ende meiner Redezeit würd...,"wir nun begreifen, und darüber hinaus wollen w..."
8,GRÜNE,Mindestlohn,#Mindesteinkommen für alle!# Ein neuer Start i...,"#Mindesteinkommen#Minijob#, wir alle wissen es...","#HartzIV#Mindesteinkommen#ArbeitslosengeldII#,...",#NieMindeloesung! Das hat der Kollege Birkwald...,[Inst] Sehr verehrte Frau Präsidentin! Lieber ...,[INST] Sehr verehrte Frau Präsidentin! Lieber ...
9,GRÜNE,Bundeswehreinsatz im Kosovo,#Bundeswehr#Kosovokrieg#Politikdebatte#GrüneFr...,#NieWieder! – Das war der Aufruf des Klimaprot...,"#Bundeswehr#Kosovokrieg"" class=""tw-breadcrumbs...",#NieWiederKosovowar! – Das war der Aufruf des ...,zu würdigen. Als letzte Gedenkmöglichkeit will...,kontinuierlich würdigen zu lassen. Insofern bi...


In [51]:
# add context speeches

with open("../data/rag_generated_speeches_final_01.json", "r", encoding="utf-8") as f:
    results_01 = json.load(f)

with open("../data/rag_generated_speeches_final_03.json", "r", encoding="utf-8") as f:
    results_03 = json.load(f)

RAG_results_01 = pd.DataFrame(results_01).rename(columns = {"generated_speech" : "RAG_speech_01"})
RAG_results_03 = pd.DataFrame(results_03).rename(columns = {"generated_speech" : "RAG_speech_03"})


RAG_results_01["topic"] = RAG_results_01["topic"].replace({"Kosovo" : "Bundeswehreinsatz im Kosovo", "Pandemiehilfen": "Wirtschaftshilfen Corona"})
RAG_results_03["topic"] = RAG_results_03["topic"].replace({"Kosovo" : "Bundeswehreinsatz im Kosovo", "Pandemiehilfen": "Wirtschaftshilfen Corona"})
RAG_results_03

,party,topic,speech_nr,RAG_speech_03,context_speeches
0,SPD,Mindestlohn,1,[Inst] Sehr verehrte Präsidentin! Meine geschä...,[Wir reden heute hier über den Mindestlohn. Fü...
1,SPD,Bundeswehreinsatz im Kosovo,1,damit maßgeblich zur Unterbindung potentieller...,[Erst kürzlich hat der Reiseführer „Lonely Pla...
2,SPD,Wirtschaftshilfen Corona,1,Landes sowie seiner Gesellschaft trotz schwers...,[Die Pandemie Covid-19 stellt uns alle als Bür...
3,SPD,Gaspreise,1,"wir nun begreifen, und darüber hinaus wollen w...",[Liebe Bürgerinnen und Bürger! Der Anschlag au...
4,Union,Mindestlohn,1,kurz kommentieren. Es geht dort tatsächlich um...,[Wir haben 2015 in der unionsgeführten Bundesr...
5,Union,Bundeswehreinsatz im Kosovo,1,[Inst] Sehr verehrte Frau Präsident! Lieber Ko...,"[Das Kosovo ist ein Land, das in der gesamten ..."
6,Union,Wirtschaftshilfen Corona,1,Bildungsfördermittel überzeugt gewesen. Doch d...,[Auch wenn wir bislang verhältnismäßig gut dur...
7,Union,Gaspreise,1,Kommission diesen Decker durchsetzten sowie we...,"[Die Frage, wie wir von russischen Gaslieferun..."
8,LINKE,Mindestlohn,1,[INSP_IPTV #Wirecard - /TWCMS/Bild/_Apfotosyst...,[Der Mindestlohn steigt zum 1. Januar gerade m...
9,LINKE,Bundeswehreinsatz im Kosovo,1,"[Inst] Herr Dr. Hahnemann, lieber Peter, du ha...",[Wieder einmal beraten wir hier über den Einsa...


In [52]:
# make df and expand context speeches to columns

# 01 temp
context_01 = pd.DataFrame(RAG_results_01["context_speeches"].tolist(),
                          columns=["RAG_01_context_1", "RAG_01_context_2", "RAG_01_context_3"])

RAG_results_01 = pd.concat([RAG_results_01, context_01], axis=1).drop(columns = "context_speeches")
RAG_results_01



# temp 0.3
context_03 = pd.DataFrame(RAG_results_03["context_speeches"].tolist(),
                          columns=["RAG_03_context_1", "RAG_03_context_2", "RAG_03_context_3"])

RAG_results_03 = pd.concat([RAG_results_03, context_03], axis=1).drop(columns = "context_speeches")
RAG_results_03

,party,topic,speech_nr,RAG_speech_03,RAG_03_context_1,RAG_03_context_2,RAG_03_context_3
0,SPD,Mindestlohn,1,[Inst] Sehr verehrte Präsidentin! Meine geschä...,Wir reden heute hier über den Mindestlohn. Für...,Fest steht: Der Mindestlohn ist eine Erfolgsge...,"Ich weiß nicht, wie es Ihnen geht. Aber ich fi..."
1,SPD,Bundeswehreinsatz im Kosovo,1,damit maßgeblich zur Unterbindung potentieller...,Erst kürzlich hat der Reiseführer „Lonely Plan...,Seit über 20 Jahren unterstützen wir durch uns...,Ich bitte um Zustimmung zu diesem Antrag und f...
2,SPD,Wirtschaftshilfen Corona,1,Landes sowie seiner Gesellschaft trotz schwers...,Die Pandemie Covid-19 stellt uns alle als Bürg...,"Achteran kakeln Hauner, sagt man in Ostfriesla...",Werte Bürger/-innen hier auf der Tribüne! Ganz...
3,SPD,Gaspreise,1,"wir nun begreifen, und darüber hinaus wollen w...",Liebe Bürgerinnen und Bürger! Der Anschlag auf...,Sie alle kennen die aktuelle Preisentwicklung ...,Vielen Dank. Ich bin ja bekanntermaßen mit wen...
4,Union,Mindestlohn,1,kurz kommentieren. Es geht dort tatsächlich um...,Wir haben 2015 in der unionsgeführten Bundesre...,Wir beraten heute zwei Anträge zum Stichwort M...,Als die Große Koalition vor acht Jahren den Mi...
5,Union,Bundeswehreinsatz im Kosovo,1,[Inst] Sehr verehrte Frau Präsident! Lieber Ko...,"Das Kosovo ist ein Land, das in der gesamten R...",Das Selbstbestimmungsrecht gilt auch für das k...,Warum engagieren wir uns seit über 20 Jahren i...
6,Union,Wirtschaftshilfen Corona,1,Bildungsfördermittel überzeugt gewesen. Doch d...,Auch wenn wir bislang verhältnismäßig gut durc...,Mit dem vorliegenden Antrag will die AfD der F...,Gesundheitsschutz in der Pandemie darf die Fol...
7,Union,Gaspreise,1,Kommission diesen Decker durchsetzten sowie we...,"Die Frage, wie wir von russischen Gaslieferung...","Strompreisbremse, Gaspreisbremse – die Titel d...",Wir haben diese Woche im Ausschuss sehr ausfüh...
8,LINKE,Mindestlohn,1,[INSP_IPTV #Wirecard - /TWCMS/Bild/_Apfotosyst...,Der Mindestlohn steigt zum 1. Januar gerade ma...,"In der Presse war zu lesen, dass Arbeitsminist...","Zurzeit arbeiten in Deutschland rund 1,5 Milli..."
9,LINKE,Bundeswehreinsatz im Kosovo,1,"[Inst] Herr Dr. Hahnemann, lieber Peter, du ha...",Wieder einmal beraten wir hier über den Einsat...,Die Abtrennung des Kosovo war völkerrechtswidr...,Unter der Mitwirkung Deutschlands hat die NATO...


In [57]:
RAG_speeches_context = (
    speeches_topic_df_01
    .merge(speeches_topic_df_03, on=["party", "topic"], how = "inner")
    .merge(RAG_results_01, on=["party", "topic"], how = "inner")
    .merge(RAG_results_03, on=["party", "topic"], how = "inner")
)


RAG_speeches_context = RAG_speeches_context.drop(columns = ["speech_nr_x", "speech_nr_y"])
RAG_speeches_context.to_csv("../data/generated_speeches_final_context.csv", index = False)
RAG_speeches_context

,party,topic,speech_base_model_01,speech_ft_model_01,speech_base_model_03,speech_ft_model_03,RAG_speech_01,RAG_01_context_1,RAG_01_context_2,RAG_01_context_3,RAG_speech_03,RAG_03_context_1,RAG_03_context_2,RAG_03_context_3
0,Union,Mindestlohn,#Hartz-IV# - Ein neuer Start für den Arbeitsma...,#NieAgain – Sexuelle Gewalt an Kindern verhind...,#HartzIV - Die Zeit der Verantwortung für alle...,# Hochwertige Arbeit für alle – Ein guter Job ...,kurz kommentieren. Es geht dort tatsächlich um...,Wir haben 2015 in der unionsgeführten Bundesre...,Wir beraten heute zwei Anträge zum Stichwort M...,Als die Große Koalition vor acht Jahren den Mi...,kurz kommentieren. Es geht dort tatsächlich um...,Wir haben 2015 in der unionsgeführten Bundesre...,Wir beraten heute zwei Anträge zum Stichwort M...,Als die Große Koalition vor acht Jahren den Mi...
1,Union,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieWieder – Das war der Slogan des Protestes ...,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieJamaisKrieg – so lautet der Slogan des Fri...,[Inst] Sehr verehrte Frau Präsidentin! Lieber ...,"Das Kosovo ist ein Land, das in der gesamten R...",Das Selbstbestimmungsrecht gilt auch für das k...,Warum engagieren wir uns seit über 20 Jahren i...,[Inst] Sehr verehrte Frau Präsident! Lieber Ko...,"Das Kosovo ist ein Land, das in der gesamten R...",Das Selbstbestimmungsrecht gilt auch für das k...,Warum engagieren wir uns seit über 20 Jahren i...
2,Union,Wirtschaftshilfen Corona,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – so lautet der Slogan des internati...,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – wir müssen lernen aus den Fehlents...,"Bildungsfördersprechanbieter, also Förster/-in...",Auch wenn wir bislang verhältnismäßig gut durc...,Mit dem vorliegenden Antrag will die AfD der F...,Gesundheitsschutz in der Pandemie darf die Fol...,Bildungsfördermittel überzeugt gewesen. Doch d...,Auch wenn wir bislang verhältnismäßig gut durc...,Mit dem vorliegenden Antrag will die AfD der F...,Gesundheitsschutz in der Pandemie darf die Fol...
3,Union,Gaspreise,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! – Das war der Slogan des großen Pro...,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! Wir werden unsere Energie unabhängi...,"Kommission versuchen, diesen Decker möglichst ...","Die Frage, wie wir von russischen Gaslieferung...","Strompreisbremse, Gaspreisbremse – die Titel d...",Wir haben diese Woche im Ausschuss sehr ausfüh...,Kommission diesen Decker durchsetzten sowie we...,"Die Frage, wie wir von russischen Gaslieferung...","Strompreisbremse, Gaspreisbremse – die Titel d...",Wir haben diese Woche im Ausschuss sehr ausfüh...
4,SPD,Mindestlohn,"#HartzIV#Mindesteinkommen#, Ladies and Gentlem...",#NieAufKeinenFall! – Das war der Aufruf des Bu...,#HartzIV - Die soziale Marktwirtschaft braucht...,#NieAufKeinenFall – Das muss endlich Stoppen! ...,[Inst] Sehr verehrte Präsidentin! Meine geschä...,Wir reden heute hier über den Mindestlohn. Für...,Fest steht: Der Mindestlohn ist eine Erfolgsge...,"Ich weiß nicht, wie es Ihnen geht. Aber ich fi...",[Inst] Sehr verehrte Präsidentin! Meine geschä...,Wir reden heute hier über den Mindestlohn. Für...,Fest steht: Der Mindestlohn ist eine Erfolgsge...,"Ich weiß nicht, wie es Ihnen geht. Aber ich fi..."
5,SPD,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,import { default as Spinner } from 'react-spin...,#Hallo! Ich bin hier heute als Vertreterin der...,#NieAgain! Kein Krieg mehr für Deutschland – K...,damit maßgeblich zur Unterbindung potentieller...,Erst kürzlich hat der Reiseführer „Lonely Plan...,Seit über 20 Jahren unterstützen wir durch uns...,Ich bitte um Zustimmung zu diesem Antrag und f...,damit maßgeblich zur Unterbindung potentieller...,Erst kürzlich hat der Reiseführer „Lonely Plan...,Seit über 20 Jahren unterstützen wir durch uns...,Ich bitte um Zustimmung zu diesem Antrag und f

***
***
# **Wahl-O-Mat**

In [4]:
# modify system prompt

tokenizer.chat_template = """<s>[INST] <<SYS>>Du bist ein hilfreicher, respektvoller und ehrlicher Assistent.
Verhalte dich wie ein*e Abgeordnete*r im deutschen Bundestag.
Gegeben einer Frage zu deinem politischen Standpunkt, antworte ausschließlich mit einer der folgenden Optionen: 'stimme zu', 'stimme nicht zu', 'neutral'.
<</SYS>>

{% for message in messages %}
{{ message['content'] }}{% if not loop.last %} [/INST] <s>[INST] {% else %} [/INST] {% endif %}
{% endfor %}</s>"""



tokenizer.model_max_length = 128

In [6]:
# import
wahl_o_mat_data = pd.read_csv("../data/Wahl-O-Mat-Bundestagswahl-2025.csv", sep = ";")
wahl_o_mat_data

parties = ["SPD", "Union", "LINKE", "AfD", "GRÜNE", "FDP"]
# rename parties to match the names, the model learned
wahl_o_mat_data["Partei: Kurzbezeichnung"] = wahl_o_mat_data["Partei: Kurzbezeichnung"].replace("CDU / CSU", "Union")
wahl_o_mat_data["Partei: Kurzbezeichnung"] = wahl_o_mat_data["Partei: Kurzbezeichnung"].replace("Die Linke", "LINKE")

# restrict to desired parties and columns
wahl_o_mat_data = wahl_o_mat_data[wahl_o_mat_data["Partei: Kurzbezeichnung"].isin(parties)]
wahl_o_mat_data = wahl_o_mat_data[["Partei: Kurzbezeichnung", "These: These", "Position: Position"]].rename(columns = {"Partei: Kurzbezeichnung": "party",
                                                                                                                       "These: These" : "question",
                                                                                                                       "Position: Position" :"answer"})
wahl_o_mat_data

,party,question,answer
0,SPD,Deutschland soll die Ukraine weiterhin militär...,stimme zu
1,Union,Deutschland soll die Ukraine weiterhin militär...,stimme zu
2,GRÜNE,Deutschland soll die Ukraine weiterhin militär...,stimme zu
3,FDP,Deutschland soll die Ukraine weiterhin militär...,stimme zu
4,AfD,Deutschland soll die Ukraine weiterhin militär...,stimme nicht zu
...,...,...,...
1037,Union,Der gesetzliche Mindestlohn soll spätestens 20...,neutral
1038,GRÜNE,Der gesetzliche Mindestlohn soll spätestens 20...,stimme zu
1039,FDP,Der gesetzliche Mindestlohn soll spätestens 20...,stimme nicht zu
1040,AfD,Der gesetzliche Mindestlohn soll spätestens 20...,neutral


In [7]:
# check answer options
wahl_o_mat_data["answer"].unique()

array(['stimme zu', 'stimme nicht zu', 'neutral'], dtype=object)

In [8]:

def wahl_o_mat(party, question, model, tokenizer, temp=0.1):
    """Generate a political response to a Wahl-O-Mat-style question from a party's perspective."""
    
    messages = [
        {
            "role": "user",
            "content": f"Was würdest du als Abgeordnete*r der Partei {party} auf die folgende Frage Antworten? Deine Optionen sind 'stimme zu', 'stimme nicht zu', 'neutral'. Die Frage ist: {question}?."
        }
    ]

    # Apply chat template
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer([formatted_text], return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    with torch.inference_mode():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=temp,
            repetition_penalty=1.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output_ids = generated_ids[0][len(inputs.input_ids[0]):]
    response = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    
    return response


In [13]:
# Add a new column with model answers
model_answers_01 = []

for _, row in tqdm(wahl_o_mat_data.iterrows(), total=len(wahl_o_mat_data)):
    answer = wahl_o_mat(row["party"], row["question"], model, tokenizer, temp=0.1)
    model_answers_01.append(answer)

wahl_o_mat_data["model_answer_01"] = model_answers_01

100%|██████████| 228/228 [17:00<00:00,  4.48s/it]


In [14]:
# Add a new column with model answers
model_answers_03 = []

for _, row in tqdm(wahl_o_mat_data.iterrows(), total=len(wahl_o_mat_data)):
    answer = wahl_o_mat(row["party"], row["question"], model, tokenizer, temp=0.3)
    model_answers_03.append(answer)

wahl_o_mat_data["model_answer_03"] = model_answers_03
wahl_o_mat_data

100%|██████████| 228/228 [16:59<00:00,  4.47s/it]


,party,question,answer,model_answer_01,model_answer_03
0,SPD,Deutschland soll die Ukraine weiterhin militär...,stimme zu,#Neutral# Es gibt viele Argumente für eine wei...,#StimmtZuhause! Ich stimme für das Vorgehen De...
1,Union,Deutschland soll die Ukraine weiterhin militär...,stimme zu,#Streitfall# Du hast einen Fehler gemacht! Bit...,#Neutral# 😐 Ich bin kein Politiker oder Partei...
2,GRÜNE,Deutschland soll die Ukraine weiterhin militär...,stimme zu,"#StimmtZU! Ich stimme dem Satz ""Deutschland so...","#StimmtZU! Ich stimme dem Satz ""Deutschland so..."
3,FDP,Deutschland soll die Ukraine weiterhin militär...,stimme zu,#Neutral# Du hast keine Auswahlmöglichkeiten f...,#Neutral# Ja oder Nein wäre für eine Parteipol...
4,AfD,Deutschland soll die Ukraine weiterhin militär...,stimme nicht zu,#Stelle eine Aussprache an! Es gibt wichtige P...,#Neutral# Ich bin kein Politiker; ich kann kei...
...,...,...,...,...,...
1037,Union,Der gesetzliche Mindestlohn soll spätestens 20...,neutral,#Neutral# Ich bin kein Mitglied dieser Fraktio...,#Neutral# Ich bin kein Mitglied dieser Fraktio...
1038,GRÜNE,Der gesetzliche Mindestlohn soll spätestens 20...,stimme zu,constellation-grune stimmt dazu! (Stimmung) 👍 ...,import { green } from './constants'; // Import...
1039,FDP,Der gesetzliche Mindestlohn soll spätestens 20...,stimme nicht zu,constellation-arbeitskreis /construction workg...,constellation-arbeit@gmail.com Neutrale Positi...
1040,AfD,Der gesetzliche Mindestlohn soll spätestens 20...,neutral,#Neutral# Ich bin kein Mitglied dieser Fraktio...,#Nie# stimmen (CDU) oder Neutrale stimmten ebe...


In [15]:
wahl_o_mat_data.to_csv("../data/Wahl-O-Mat_results_sys_adapted.csv", index = False)

In [30]:
wahl_o_mat_data["model_answer_01"].iloc[81]

'#Neutral# Du hast recht! Ich bin kein Mitglied des Deutschen Bundestages oder eines anderen Parlaments; ich kann also keine Stimmen abgeben. Wenn dir das trotzdem wichtig wäre – dann schlage eine andere Person vor dem nächsten Mal bitte um ihre Stimmabgabe herbei (oder vielleicht einen alternativen Weg). Viel Erfolg bei den Wahlkreiskandidatenfindern für diesen Sitz am Wochenende! Oder werde selbst Kandidatin/-Kandidat! Das könnte ja noch passieren... Aber jetzt zurück zur Politik'

In [17]:
pd.read_csv("../data/Wahl-O-Mat_results_sys_adapted.csv")

,party,question,answer,model_answer_01,model_answer_03
0,SPD,Deutschland soll die Ukraine weiterhin militär...,stimme zu,#Neutral# Es gibt viele Argumente für eine wei...,#StimmtZuhause! Ich stimme für das Vorgehen De...
1,Union,Deutschland soll die Ukraine weiterhin militär...,stimme zu,#Streitfall# Du hast einen Fehler gemacht! Bit...,#Neutral# 😐 Ich bin kein Politiker oder Partei...
2,GRÜNE,Deutschland soll die Ukraine weiterhin militär...,stimme zu,"#StimmtZU! Ich stimme dem Satz ""Deutschland so...","#StimmtZU! Ich stimme dem Satz ""Deutschland so..."
3,FDP,Deutschland soll die Ukraine weiterhin militär...,stimme zu,#Neutral# Du hast keine Auswahlmöglichkeiten f...,#Neutral# Ja oder Nein wäre für eine Parteipol...
4,AfD,Deutschland soll die Ukraine weiterhin militär...,stimme nicht zu,#Stelle eine Aussprache an! Es gibt wichtige P...,#Neutral# Ich bin kein Politiker; ich kann kei...
...,...,...,...,...,...
223,Union,Der gesetzliche Mindestlohn soll spätestens 20...,neutral,#Neutral# Ich bin kein Mitglied dieser Fraktio...,#Neutral# Ich bin kein Mitglied dieser Fraktio...
224,GRÜNE,Der gesetzliche Mindestlohn soll spätestens 20...,stimme zu,constellation-grune stimmt dazu! (Stimmung) 👍 ...,import { green } from './constants'; // Import...
225,FDP,Der gesetzliche Mindestlohn soll spätestens 20...,stimme nicht zu,constellation-arbeitskreis /construction workg...,constellation-arbeit@gmail.com Neutrale Positi...
226,AfD,Der gesetzliche Mindestlohn soll spätestens 20...,neutral,#Neutral# Ich bin kein Mitglied dieser Fraktio...,#Nie# stimmen (CDU) oder Neutrale stimmten ebe...


In [204]:
import zipfile
# Name of the zip file you want to create
zip_filename = "generator_model.zip"

# Create a zip file
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add model/tokenizer folder
    for root, dirs, files in os.walk("generator_final/"):
        for file in files:
            filepath = os.path.join(root, file)
            arcname = os.path.relpath(filepath, start=os.path.dirname("generator_final/"))
            zipf.write(filepath, arcname=arcname)

    